<a href="https://colab.research.google.com/github/lachlandeer/2016-04-07-FederalReserveBoard/blob/FirstLoad/user_gender_testset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sat Dec 19 22:53:01 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    24W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from google.colab import auth
auth.authenticate_user()

# Get Data, Model and Tokenizer From Google Storage¶


In [4]:
project_id = 'egypt-riot-network'


In [5]:
!gcloud config set project {project_id}
!gsutil ls

Updated property [core/project].


To take a quick anonymous survey, run:
  $ gcloud survey

gs://egypt-riot-network/


In [6]:
!mkdir data model tokenizer


In [7]:
!gsutil cp gs://egypt-riot-network/out/data/user_gender/classified_gender_test.csv   data/test_tweets.csv

Copying gs://egypt-riot-network/out/data/user_gender/classified_gender_test.csv...
- [1 files][255.4 MiB/255.4 MiB]                                                
Operation completed over 1 objects/255.4 MiB.                                    


In [9]:
# will need to change this - accidently overwrote the pa classifier :/
!gsutil cp gs://egypt-riot-network/out/models/user_pa_classifier/model/model/* model/.


Copying gs://egypt-riot-network/out/models/user_pa_classifier/model/model/config.json...
Copying gs://egypt-riot-network/out/models/user_pa_classifier/model/model/pytorch_model.bin...
- [2 files][515.8 MiB/515.8 MiB]                                                
Operation completed over 2 objects/515.8 MiB.                                    


In [10]:
!gsutil cp gs://egypt-riot-network/out/models/user_pa_classifier/tokenizer/tokenizer/* tokenizer/.


Copying gs://egypt-riot-network/out/models/user_pa_classifier/tokenizer/tokenizer/special_tokens_map.json...
Copying gs://egypt-riot-network/out/models/user_pa_classifier/tokenizer/tokenizer/tokenizer_config.json...
Copying gs://egypt-riot-network/out/models/user_pa_classifier/tokenizer/tokenizer/vocab.txt...
- [3 files][638.8 KiB/638.8 KiB]                                                
Operation completed over 3 objects/638.8 KiB.                                    


# Load data to attach predictions to


In [11]:
df = pd.read_csv('data/test_tweets.csv')


In [12]:
df.head()


,user_id,id,text_seg,gender
0,1000437560,306195358747328512,[مستخدم] كلما +ت ك+ ليس +ت ك+ ال+ كلم +ات هي ك...,male
1,1000437560,295864428795138050,اح +نا +كن +نا ب+ نطالب ب+ مصالح +ات جنوب افري...,male
2,1000437560,295864097239597058,مصر ال+ يوم تحترق,male
3,1000437560,283615531507019777,ف+ سوف يستفتى باقي ال+ شعب علي تشكيل يخالف ال+...,male
4,1000437560,283615278863101952,حال كون مجلس ال+ شورى ب+ تشكيل +ه ال+ حالي قبل...,male


In [13]:
DATA_COLUMN = 'text'
ID_COLUMN   = 'id'

df_test = df[['id', 'text_seg']]
df_test.columns = [ID_COLUMN, DATA_COLUMN]


In [15]:
df_test.head()

,id,text
0,306195358747328512,[مستخدم] كلما +ت ك+ ليس +ت ك+ ال+ كلم +ات هي ك...
1,295864428795138050,اح +نا +كن +نا ب+ نطالب ب+ مصالح +ات جنوب افري...
2,295864097239597058,مصر ال+ يوم تحترق
3,283615531507019777,ف+ سوف يستفتى باقي ال+ شعب علي تشكيل يخالف ال+...
4,283615278863101952,حال كون مجلس ال+ شورى ب+ تشكيل +ه ال+ حالي قبل...


In [16]:
!mkdir out_data
df_test.to_csv("out_data/test.tsv",index=False,columns=df_test.columns,sep='\t',header=False)

# Imports

In [17]:
!pip install farasapy
!pip install pyarabic
!git clone https://github.com/aub-mind/arabert


     |████████████████████████████████| 112kB 13.8MB/s 
  Created wheel for pyarabic: filename=PyArabic-0.6.10-cp36-none-any.whl size=113324 sha256=26d05ea45fc846f8e1f947191f0ffc9c8d8776c17e1a607f706388f622b3958e
  Stored in directory: /root/.cache/pip/wheels/10/b8/f5/b7c1a50e6efb83544844f165a9b134afe7292585465e29b61d
Successfully built pyarabic
Cloning into 'arabert'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 279 (delta 34), reused 38 (delta 15), pack-reused 214
Receiving objects: 100% (279/279), 3.68 MiB | 965.00 KiB/s, done.
Resolving deltas: 100% (157/157), done.


In [18]:
!pip install torch
!pip install transformers

     |████████████████████████████████| 1.5MB 13.4MB/s 
     |████████████████████████████████| 890kB 62.7MB/s 
     |████████████████████████████████| 2.9MB 49.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=700df6bb6430407b8a262749cb4e5811584d57320fd26bb1d460f590769303db
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [19]:

from __future__ import absolute_import, division, print_function

import glob
import logging
import os
import random
import json
import csv
import sys
from io import open
from multiprocessing import Pool, cpu_count
from tqdm import tqdm
import numpy as np

import torch
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm_notebook, trange
from torch.utils.tensorboard import SummaryWriter
from torch import nn
from torch.nn import CrossEntropyLoss, MSELoss

from transformers import BertPreTrainedModel
from transformers import BertModel
from transformers import WEIGHTS_NAME, BertConfig, BertForSequenceClassification, BertTokenizer                                  
from transformers import AdamW, get_linear_schedule_with_warmup

from arabert.preprocess_arabert import never_split_tokens

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [20]:
from transformers import BertForSequenceClassification


# Data Loading Functions

In [22]:

# Copyright 2018 The Google AI Language Team Authors and The HuggingFace Inc. team.
# Copyright (c) 2018, NVIDIA CORPORATION.  All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
""" BERT classification fine-tuning: utilities to work with GLUE tasks """
csv.field_size_limit(2147483647)

class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, label=None):
        """Constructs a InputExample.

        Args:
            guid: Unique id for the example.
            text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
            text_b: (Optional) string. The untokenized text of the second sequence.
            Only must be specified for sequence pair tasks.
            label: (Optional) string. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label


class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_id):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id


class DataProcessor(object):
    """Base class for data converters for sequence classification data sets."""

    def get_train_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the dev set."""
        raise NotImplementedError()

    def get_predict_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the dev set."""
        raise NotImplementedError()

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

    @classmethod
    def _read_tsv(cls, input_file, quotechar=None):
        """Reads a tab separated value file."""
        with open(input_file, "r", encoding="utf-8-sig") as f:
            reader = csv.reader(f, delimiter="\t", quotechar=quotechar)
            lines = []
            for line in reader:
                if sys.version_info[0] == 2:
                    line = list(unicode(cell, 'utf-8') for cell in line)
                lines.append(line)
            return lines


class BinaryProcessor(DataProcessor):
    """Processor for the binary data sets"""

    def get_train_examples(self, data_dir , train_file_name):
        """See base class. file should be a tsv"""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, train_file_name)), "train")

    def get_dev_examples(self, data_dir, dev_file_name):
        """See base class. file should be a tsv"""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, dev_file_name)), "dev")

    def get_predict_examples(self, data_dir, train_file_name):
        """See base class. file should be a tsv"""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, train_file_name)), "predict")

    def get_labels(self):
        """See base class."""
        return ["0", "1"]

    def _create_examples(self, lines, set_type):
        """Creates examples for the training and dev sets."""
        examples = []

        if(set_type!="predict"):
            for (i, line) in enumerate(lines):
                guid = "%s-%s" % (set_type, i)
                text_a = line[3]
                label = line[1]
                examples.append(
                    InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
        else:
            for (i, line) in enumerate(lines):
                guid = "%s-%s" % (set_type, i)
                text_a = line[1]
                label = '0'
                examples.append(
                    InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
        return examples


def convert_example_to_feature(example_row, pad_token=0,
sequence_a_segment_id=0, sequence_b_segment_id=1,
cls_token_segment_id=1, pad_token_segment_id=0,
mask_padding_with_zero=True, sep_token_extra=False):
    example, label_map, max_seq_length, tokenizer, output_mode, cls_token_at_end, cls_token, sep_token, cls_token_segment_id, pad_on_left, pad_token_segment_id, sep_token_extra = example_row

    tokens_a = tokenizer.tokenize(example.text_a)

    tokens_b = None
    if example.text_b:
        tokens_b = tokenizer.tokenize(example.text_b)
        # Modifies `tokens_a` and `tokens_b` in place so that the total
        # length is less than the specified length.
        # Account for [CLS], [SEP], [SEP] with "- 3". " -4" for RoBERTa.
        special_tokens_count = 4 if sep_token_extra else 3
        _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - special_tokens_count)
    else:
        # Account for [CLS] and [SEP] with "- 2" and with "- 3" for RoBERTa.
        special_tokens_count = 3 if sep_token_extra else 2
        if len(tokens_a) > max_seq_length - special_tokens_count:
            tokens_a = tokens_a[:(max_seq_length - special_tokens_count)]

    # The convention in BERT is:
    # (a) For sequence pairs:
    #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
    #  type_ids:   0   0  0    0    0     0       0   0   1  1  1  1   1   1
    # (b) For single sequences:
    #  tokens:   [CLS] the dog is hairy . [SEP]
    #  type_ids:   0   0   0   0  0     0   0
    #
    # Where "type_ids" are used to indicate whether this is the first
    # sequence or the second sequence. The embedding vectors for `type=0` and
    # `type=1` were learned during pre-training and are added to the wordpiece
    # embedding vector (and position vector). This is not *strictly* necessary
    # since the [SEP] token unambiguously separates the sequences, but it makes
    # it easier for the model to learn the concept of sequences.
    #
    # For classification tasks, the first vector (corresponding to [CLS]) is
    # used as as the "sentence vector". Note that this only makes sense because
    # the entire model is fine-tuned.
    tokens = tokens_a + [sep_token]
    segment_ids = [sequence_a_segment_id] * len(tokens)

    if tokens_b:
        tokens += tokens_b + [sep_token]
        segment_ids += [sequence_b_segment_id] * (len(tokens_b) + 1)

    if cls_token_at_end:
        tokens = tokens + [cls_token]
        segment_ids = segment_ids + [cls_token_segment_id]
    else:
        tokens = [cls_token] + tokens
        segment_ids = [cls_token_segment_id] + segment_ids

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)


    # Zero-pad up to the sequence length.
    padding_length = max_seq_length - len(input_ids)
    if pad_on_left:
        input_ids = ([pad_token] * padding_length) + input_ids
        input_mask = ([0 if mask_padding_with_zero else 1] * padding_length) + input_mask
        segment_ids = ([pad_token_segment_id] * padding_length) + segment_ids
    else:
        input_ids = input_ids + ([pad_token] * padding_length)
        input_mask = input_mask + ([0 if mask_padding_with_zero else 1] * padding_length)
        segment_ids = segment_ids + ([pad_token_segment_id] * padding_length)

    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length

    if output_mode == "classification":
        label_id = label_map[example.label]
    elif output_mode == "regression":
        label_id = float(example.label)
    else:
        raise KeyError(output_mode)

    return InputFeatures(input_ids=input_ids,
                        input_mask=input_mask,
                        segment_ids=segment_ids,
                        label_id=label_id)
  

def convert_examples_to_features(examples, label_list, max_seq_length,
                                 tokenizer, output_mode,
                                 cls_token_at_end=False, sep_token_extra=False, pad_on_left=False,
                                 cls_token='[CLS]', sep_token='[SEP]', pad_token=0,
                                 sequence_a_segment_id=0, sequence_b_segment_id=1,
                                 cls_token_segment_id=1, pad_token_segment_id=0,
                                 mask_padding_with_zero=True):
    """ Loads a data file into a list of `InputBatch`s
        `cls_token_at_end` define the location of the CLS token:
            - False (Default, BERT/XLM pattern): [CLS] + A + [SEP] + B + [SEP]
            - True (XLNet/GPT pattern): A + [SEP] + B + [SEP] + [CLS]
        `cls_token_segment_id` define the segment id associated to the CLS token (0 for BERT, 2 for XLNet)
    """

    label_map = {label : i for i, label in enumerate(label_list)}

    examples = [(example, label_map, max_seq_length, tokenizer, output_mode, cls_token_at_end, cls_token, sep_token, cls_token_segment_id, pad_on_left, pad_token_segment_id, sep_token_extra) for example in examples]

    process_count = cpu_count()

    with Pool(process_count) as p:
        features = list(tqdm(p.imap(convert_example_to_feature, examples, chunksize=500), total=len(examples)))

    return features


def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()


processors = {
    "binary": BinaryProcessor
}

output_modes = {
    "binary": "classification"
}

In [23]:
def load_and_cache_examples(task, tokenizer, mode="train"):
    processor = processors[task]()
    output_mode = args['output_mode']
    
    cached_features_file = os.path.join(args['data_dir'], f"cached_{mode}_{args['cache_dir']}_{args['max_seq_length']}_{task}")
    
    if os.path.exists(cached_features_file) and not args['reprocess_input_data']:
        logger.info("Loading features from cached file %s", cached_features_file)
        features = torch.load(cached_features_file)
               
    else:
        logger.info("Creating features from dataset file at %s", args['data_dir'])
        label_list = processor.get_labels()
        if mode=="train":
          examples=processor.get_train_examples(args['data_dir'],args['train_file_name']);
        if mode=="dev":
          examples=processor.get_dev_examples(args['data_dir'],args['dev_file_name']);
        if mode=="predict":
          examples=processor.get_predict_examples(args['data_dir'],args['pred_file_name'])
        
        if __name__ == "__main__":
            features = convert_examples_to_features(examples, label_list, args['max_seq_length'], tokenizer, output_mode,
                cls_token_at_end=bool(args['model_type'] in ['xlnet']),            # xlnet has a cls token at the end
                cls_token=tokenizer.cls_token,
                cls_token_segment_id=2 if args['model_type'] in ['xlnet'] else 0,
                sep_token=tokenizer.sep_token,
                sep_token_extra=bool(args['model_type'] in ['roberta']),           # roberta uses an extra separator b/w pairs of sentences, cf. github.com/pytorch/fairseq/commit/1684e166e3da03f5b600dbb7855cb98ddfcd0805
                pad_on_left=bool(args['model_type'] in ['xlnet']),                 # pad on the left for xlnet
                pad_token=tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0],
                pad_token_segment_id=4 if args['model_type'] in ['xlnet'] else 0)
        
        logger.info("Saving features into cached file %s", cached_features_file)
        torch.save(features, cached_features_file)
        
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
    if output_mode == "classification":
        all_label_ids = torch.tensor([f.label_id for f in features], dtype=torch.long)
    elif output_mode == "regression":
        all_label_ids = torch.tensor([f.label_id for f in features], dtype=torch.float)

    dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
    return dataset

# Predict Function

In [24]:
def predict(model, tokenizer, prefix=""):
  pred_output_dir = args['output_dir']
  
  results={}
  PRED_TASK = args['task_name']
  model.eval()
  
  pred_dataset = load_and_cache_examples(PRED_TASK, tokenizer, mode='predict')
  if not os.path.exists(pred_output_dir):
    os.makedirs(pred_output_dir)
    
  
  pred_sampler = SequentialSampler(pred_dataset)
  pred_dataloader = DataLoader(pred_dataset, sampler=pred_sampler, batch_size=args['eval_batch_size'])
  
  logger.info("***** Running prediction {} *****".format(prefix))
  logger.info("  Num examples = %d", len(pred_dataset))
  logger.info("  Batch size = %d", args['eval_batch_size'])
  
  preds = None
  for batch in pred_dataloader:
    with torch.no_grad():
      batch = tuple(t.to(device) for t in batch)
      inputs = {'input_ids': batch[0],'attention_mask': batch[1],'token_type_ids': batch[2],'labels': batch[3]}
      
      outputs = model(**inputs)
      _, logits = outputs[:2]
    if preds is None:
        preds = logits.detach().cpu().numpy()
    else:
        preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)

    #torch.cuda.empty_cache()        

  preds = np.argmax(preds, axis=1)
      
  return preds

# Define Model Params

In [25]:
args = {
    'data_dir': 'out_data/',
    'train_file_name': 'train.tsv',
    'dev_file_name': 'dev.tsv',
    'pred_file_name': 'test.tsv',
    'model_type':  'bert',
    'model_name': 'aubmindlab/bert-base-arabert',
    'task_name': 'binary',
    'output_dir': 'outputs_bert/',
    'cache_dir': 'cache',
    'do_train': False,
    'do_eval': True,
    'fp16': False,
    'fp16_opt_level': 'O1',
    'max_seq_length': 128,
    'output_mode': 'classification',
    'train_batch_size': 16,
    'eval_batch_size': 32,
    'num_train_epochs': 3,
    'weight_decay': 0,
    'learning_rate': 2e-5,
    'adam_epsilon': 1e-8,
    'warmup_steps': 0,
    'max_grad_norm': 1.0,
    'log_dir':'./logs',
    'logging_steps': 0,
    'evaluate_during_training': True,
    'save_steps': 90,
    'eval_all_checkpoints': True,
    'overwrite_output_dir': True,
    'reprocess_input_data': False,
    'notes': 'action_classification_arabert'
}
with open('args.json', 'w') as f:
    json.dump(args, f)

!mkdir ./{args['log_dir']}

# Create & Configure

In [26]:
MODEL_CLASSES = {
    'bert': (BertConfig, BertForSequenceClassification, BertTokenizer),
    #'xlnet': (XLNetConfig, XLNetForSequenceClassification, XLNetTokenizer),
    #'xlm': (XLMConfig, XLMForSequenceClassification, XLMTokenizer),
    #'roberta': (RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer)
}
config_class, model_class, tokenizer_class = MODEL_CLASSES[args['model_type']]

In [27]:
#Checking for GPU
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))
    !nvidia-smi

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-SXM2-16GB
Sat Dec 19 23:12:50 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    24W / 300W |     10MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                

# Load Tokenizer and Model From File

In [28]:
#https://github.com/huggingface/transformers/blob/master/src/transformers/configuration_bert.py#L52
#config = config_class.from_pretrained(args['model_name'], num_labels=2, finetuning_task=args['task_name'])

model = model_class.from_pretrained('model/', output_attentions=True)
model.to(device)

#https://github.com/huggingface/transformers/blob/master/src/transformers/tokenization_bert.py#L119
tokenizer = tokenizer_class.from_pretrained('tokenizer/',
    do_lower_case=False,
    do_basic_tokenize=True,
    never_split=never_split_tokens)

In [29]:
task = args['task_name']

if task in processors.keys() and task in output_modes.keys():
    processor = processors[task]()
    label_list = processor.get_labels()
    num_labels = len(label_list)
else:
    raise KeyError(f'{task} not found in processors or in output_modes. Please check utils.py.')

# Predict

In [30]:
preds = predict(model,tokenizer)


INFO:__main__:Creating features from dataset file at out_data/
100%|██████████| 1404013/1404013 [05:22<00:00, 4351.73it/s]
INFO:__main__:Saving features into cached file out_data/cached_predict_cache_128_binary
INFO:__main__:***** Running prediction  *****
INFO:__main__:  Num examples = 1404013
INFO:__main__:  Batch size = 32


Join Predictions to Tweets

In [31]:
df_predictions = pd.concat((df.reset_index(),
                            pd.DataFrame(preds,columns=['predicted'])
                            ),
                           axis=1,
                           ignore_index=False
                           )

In [32]:
df_predictions.head(n = 20)


,index,user_id,id,text_seg,gender,predicted
0,0,1000437560,306195358747328512,[مستخدم] كلما +ت ك+ ليس +ت ك+ ال+ كلم +ات هي ك...,male,0
1,1,1000437560,295864428795138050,اح +نا +كن +نا ب+ نطالب ب+ مصالح +ات جنوب افري...,male,1
2,2,1000437560,295864097239597058,مصر ال+ يوم تحترق,male,0
3,3,1000437560,283615531507019777,ف+ سوف يستفتى باقي ال+ شعب علي تشكيل يخالف ال+...,male,0
4,4,1000437560,283615278863101952,حال كون مجلس ال+ شورى ب+ تشكيل +ه ال+ حالي قبل...,male,0
5,5,1000437560,283614855443935232,و+ مع ال+ تسليم جدل +ا بان ال+ عبر +ة ب+ عرض +...,male,0
6,6,1000437560,283614603861184512,ال+ حالي ف+ لا يجوز تعديل تشكيل +ه بعد اعداد م...,male,1
7,7,1000437560,283614455911305216,على أن يتم ذلك خلال سن +ة من تاريخ انعقاد مجلس...,male,0
8,8,1000437560,283613874538815490,و+ تنتقل إلى مجلس ال+ نواب ، فور انتخاب +ه ، ا...,male,0
9,9,1000437560,283613463782252545,يتولى مجلس ال+ شورى ال+ قائم ب+ تشكيل +ه ال+ ح...,male,1


In [33]:
cols_keep = ['user_id', 'gender', 'predicted']
df_predictions = df_predictions[cols_keep]

df_predictions.head()

,user_id,gender,predicted
0,1000437560,male,0
1,1000437560,male,1
2,1000437560,male,0
3,1000437560,male,0
4,1000437560,male,0


# Recode pol affil to binary to compare to predictions¶


In [34]:
label_map = {
    'male' : 0,
    'female' : 1
}

df_predictions['gender'] = df_predictions['gender'].apply(lambda x: label_map[x])

In [35]:
#https://stackoverflow.com/questions/32459325/python-pandas-dataframe-select-row-by-max-value-in-group
user_tweet_type_count = df_predictions.groupby(['user_id', 'predicted']).agg(lambda x:x.value_counts()).reset_index()

In [36]:
user_preds = user_tweet_type_count.loc[user_tweet_type_count.reset_index().groupby(['user_id'])['gender'].idxmax()]


In [37]:
user_preds.groupby(['predicted']).size()


predicted
0    1069
1     493
dtype: int64

In [38]:
user_preds.drop(['gender'], inplace = True, axis = 1)


# Load Ground Truth Predictions

In [41]:
!gsutil cp gs://egypt-riot-network/out/data/user_gender/classified_gender_test_users.csv data/users_test.csv


Copying gs://egypt-riot-network/out/data/user_gender/classified_gender_test_users.csv...
/ [1 files][ 23.3 KiB/ 23.3 KiB]                                                
Operation completed over 1 objects/23.3 KiB.                                     


In [42]:
user_types = pd.read_csv('data/users_test.csv')


In [43]:
user_types['gender'] = user_types['gender'].apply(lambda x: label_map[x])

# Merge Predictions & Truth Together

In [44]:
user_comparison = user_types.set_index('user_id').join(user_preds.set_index('user_id'), how = 'inner')


# Classification Report

In [45]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [48]:
confusion_matrix(user_comparison['gender'], user_comparison['predicted'])


array([[1008,  285],
       [  61,  208]])

In [49]:
print(classification_report(user_comparison['gender'], user_comparison['predicted']))


              precision    recall  f1-score   support

           0       0.94      0.78      0.85      1293
           1       0.42      0.77      0.55       269

    accuracy                           0.78      1562
   macro avg       0.68      0.78      0.70      1562
weighted avg       0.85      0.78      0.80      1562



# Save Predictions Next to Truths

In [ ]:
user_comparison.to_csv('out_data/predictions_arabert_testset.csv',
                                index = False)


In [ ]:
!gsutil cp -r out_data/predictions_arabert_testset.csv gs://egypt-riot-network/out/data/user_gender
